In [5]:
import pdfminer
import pdfminer.high_level
import os
import pandas as pd

In [6]:
if not os.path.exists('done'):
    os.makedirs('done')

In [7]:
num_pages = 0
texts = []
for file in os.listdir('.'):
    if file.endswith('.pdf'):
        with open(file, 'rb') as f:
            texts.append(pdfminer.high_level.extract_text(f))
        os.rename(file, 'done/' + file)

In [8]:
# divide the text into pages
pages = []
for text in texts:
    pages += text.split('\x0c')

In [9]:
df = pd.DataFrame(pages, columns=['text'])
df = df.text.str.split('\n', expand=True)

In [10]:
df

,0,1,2,3,4,5,6,7,8,9,...,125,126,127,128,129,130,131,132,133,134
0,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
1,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
2,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
3,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
4,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
5,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
6,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
7,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
8,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,,,,
9,SALES,,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,,Address,,Aeropark - Yenişehir,,Mahallesi Osmanlı Bulvar No:11/A,,...,,,,,,,None,None,None,None


In [11]:
# change all None to empty strings
df = df.fillna('')

# replace empty strings with NaN so dropna works on them
import numpy as np
df = df.replace('', np.nan)

# drop columns and rows where all values are NaN
df = df.dropna(axis=1, how='all').reset_index(drop=True)
df = df.dropna(axis=0, how='all').reset_index(drop=True)

C:\Users\Kamil Tomaszek\AppData\Local\Temp\ipykernel_988\2845531025.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)


In [12]:
# remove illigal characters
df = df.replace(to_replace=r'\f', value='', regex=True)

In [13]:
df

,0,2,4,6,8,10,12,14,16,18,...,124,125,126,127,128,129,130,131,132,133
0,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
1,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
2,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
3,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
4,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
5,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
6,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
7,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
8,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,,,,
9,SALES,PEGASUS HAVA TAŞIMACILIĞI A.Ş.,Address,Aeropark - Yenişehir,Mahallesi Osmanlı Bulvar No:11/A,34912 Kurtköy Pendik/İSTANBUL,Central Reg. System No : 0723004708500017 -,Büyük Mükellefler V.D. : 723 004 7085,Web :,http://www.flypgs.com,...,,,,,,,NaN,NaN,NaN,NaN


In [14]:
# save to excel
df.to_excel('output.xlsx', index=False)